# Proyecto Final - Security Data Science

## Oscar Fernando López Barrios
## Carné 20679

In [12]:
# Importar librerías
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from imblearn.over_sampling import SMOTE

In [13]:
# Cargar drive para el dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Cargar el dataset
df = pd.read_csv('/content/drive/My Drive/Personal/SDS/fraud_detection_dataset.csv')

In [15]:
# Mostrar el dataframe
df

,amt,is_fraud,amt_month_shopping_net_spend,count_month_shopping_net,first_time_at_merchant,hour,minute,second,day,month,...,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,7.27,0,0.00,0.0,True,12,47,15,1,1,...,0,0,0,0,1,0,0,0,0,0
1,52.94,0,0.00,0.0,True,8,44,57,2,1,...,0,0,0,0,0,0,0,0,0,0
2,82.08,0,0.00,0.0,True,8,47,36,2,1,...,0,0,0,0,0,0,0,0,0,0
3,34.79,0,0.00,0.0,True,12,38,14,2,1,...,0,0,0,1,0,0,0,0,0,0
4,27.18,0,0.00,0.0,True,13,10,46,2,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,66.11,0,160.56,11.0,False,2,10,10,31,12,...,0,0,0,0,0,0,0,0,0,0
1852390,4.58,0,160.56,11.0,False,5,13,35,31,12,...,0,0,0,0,1,0,0,0,0,0
1852391,95.96,0,160.56,11.0,False,11,15,44,31,12,...,0,0,0,0,0,0,0,0,0,0
1852392,149.48,0,160.56,11.0,False,11,17,25,31,12,...,1,0,0,0,0,0,0,0,0,0


In [16]:
# Mostrar los tipos de datos
df.dtypes

amt                             float64
is_fraud                          int64
amt_month_shopping_net_spend    float64
count_month_shopping_net        float64
first_time_at_merchant             bool
                                 ...   
category_misc_pos                 int64
category_personal_care            int64
category_shopping_net             int64
category_shopping_pos             int64
category_travel                   int64
Length: 64, dtype: object

In [17]:
# Mostrar las cantidad de datos
fraud_counts = df['is_fraud'].value_counts()
print(fraud_counts)

is_fraud
0    1842743
1       9651
Name: count, dtype: int64


In [18]:
# Eliminar los valores nulos
df.fillna(0, inplace=True)

In [8]:
# Separar características y objetivo
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

In [9]:
# Función para entrenar el modelo por un período de tiempo específico y devolver métricas de rendimiento
def train_xgboost(X_train, y_train, X_test, y_test, model=None):
    # Escalar las características para mejorar el rendimiento del modelo
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Aplicar SMOTE
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

    # Crear el objeto DMatrix
    dtrain = xgb.DMatrix(X_train_res, label=y_train_res)
    dtest = xgb.DMatrix(X_test_scaled, label=y_test)

    # Parámetros del modelo
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': 0.05,
        'max_depth': 6,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'gamma': 0,
        'seed': 42
    }

    # Número de rondas de boosting
    num_boost_round = 100

    # Lista para almacenar las métricas de validación
    evals = [(dtest, 'eval')]

    # Lista de seguimiento del progreso
    progress = {}

    # Entrenar el modelo
    if model is None:
        model = xgb.train(params, dtrain, num_boost_round=num_boost_round, evals=evals, evals_result=progress, verbose_eval=False)
    else:
        model = xgb.train(params, dtrain, num_boost_round=num_boost_round, evals=evals, evals_result=progress, xgb_model=model, verbose_eval=False)

    # Predecir en el conjunto de prueba
    y_pred_proba = model.predict(dtest)
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Calcular métricas de rendimiento
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    conf_matrix = confusion_matrix(y_test, y_pred)

    return model, accuracy, precision, recall, f1, roc_auc, conf_matrix

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Variables para almacenar las métricas acumuladas
total_accuracy = 0
total_precision = 0
total_recall = 0
total_f1 = 0
total_roc_auc = 0
confusion_matrices = []

# Contador de períodos
period_count = 0

# Inicializar el modelo
model = None

# Loop para entrenar de manera incremental por cada 3 meses
for year in range(df['year'].min(), df['year'].max() + 1):
    for month in range(1, 13, 3):  # Entrenar cada 3 meses
        # Filtrar los datos para el año y mes actual
        X_train_period = X_train_full[(X_train_full['year'] == year) & (X_train_full['month'].between(month, month + 2))]
        y_train_period = y_train_full[X_train_period.index]

        # Asegurarse de que no hay datos vacíos
        if len(X_train_period) == 0:
            continue

        # Entrenar el modelo XGBoost para este período de tiempo
        print(f"Training for year {year}, month {month}-{month + 2}")
        model, accuracy, precision, recall, f1, roc_auc, conf_matrix = train_xgboost(X_train_period, y_train_period, X_test, y_test, model)

        # Acumular métricas
        total_accuracy += accuracy
        total_precision += precision
        total_recall += recall
        total_f1 += f1
        total_roc_auc += roc_auc
        confusion_matrices.append(conf_matrix)

        # Incrementar el contador de períodos
        period_count += 1

        # Imprimir métricas para este período
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC Score: {roc_auc:.4f}")
        print(f"Confusion Matrix:\n{conf_matrix}")

# Calcular las métricas finales promedio
final_accuracy = total_accuracy / period_count
final_precision = total_precision / period_count
final_recall = total_recall / period_count
final_f1 = total_f1 / period_count
final_roc_auc = total_roc_auc / period_count

# Sumar las matrices de confusión
final_confusion_matrix = np.sum(confusion_matrices, axis=0)

# Imprimir las métricas finales
print("\nFinal Metrics:")
print(f"Accuracy: {final_accuracy:.4f}")
print(f"Precision: {final_precision:.4f}")
print(f"Recall: {final_recall:.4f}")
print(f"F1 Score: {final_f1:.4f}")
print(f"ROC AUC Score: {final_roc_auc:.4f}")
print(f"Confusion Matrix:\n{final_confusion_matrix}")

Training for year 2019, month 1-3
Accuracy: 0.9979
Precision: 0.8007
Recall: 0.7782
F1 Score: 0.7892
ROC AUC Score: 0.9960
Confusion Matrix:
[[552318    552]
 [   632   2217]]
Training for year 2019, month 4-6
Accuracy: 0.9984
Precision: 0.8268
Recall: 0.8715
F1 Score: 0.8486
ROC AUC Score: 0.9984
Confusion Matrix:
[[552350    520]
 [   366   2483]]
Training for year 2019, month 7-9
Accuracy: 0.9984
Precision: 0.8264
Recall: 0.8621
F1 Score: 0.8438
ROC AUC Score: 0.9981
Confusion Matrix:
[[552354    516]
 [   393   2456]]
Training for year 2019, month 10-12
Accuracy: 0.9984
Precision: 0.8044
Recall: 0.9077
F1 Score: 0.8529
ROC AUC Score: 0.9983
Confusion Matrix:
[[552241    629]
 [   263   2586]]
Training for year 2020, month 1-3
Accuracy: 0.9987
Precision: 0.8961
Recall: 0.8508
F1 Score: 0.8729
ROC AUC Score: 0.9965
Confusion Matrix:
[[552589    281]
 [   425   2424]]
Training for year 2020, month 4-6
Accuracy: 0.9985
Precision: 0.8428
Recall: 0.8786
F1 Score: 0.8603
ROC AUC Score: 0.

In [10]:
# División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Balanceo de datos en el conjunto de entrenamiento utilizando SMOTE
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)



# Parámetros del modelo XGBoost
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.05,
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0,
    'seed': 42
}

# Número de rondas de boosting
num_boost_round = 100

dtrain = xgb.DMatrix(X_train_res, label=y_train_res)
dtest = xgb.DMatrix(X_test_scaled, label=y_test)

# Entrenamiento del modelo XGBoost
model = xgb.train(params, dtrain, num_boost_round=num_boost_round, evals=[(dtest, 'eval')], verbose_eval=False)

# Predicción en el conjunto de prueba
y_pred_proba = model.predict(dtest)
y_pred = (y_pred_proba > 0.5).astype(int)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcular curva precision-recall y área bajo la curva (AUC)
precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall_curve, precision_curve)

# Imprimir métricas de rendimiento
print("Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(f"PR AUC Score: {pr_auc:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Metrics:
Accuracy: 0.9952
Precision: 0.5163
Recall: 0.9807
F1 Score: 0.6764
ROC AUC Score: 0.9988
PR AUC Score: 0.9600
Confusion Matrix:
[[550252   2618]
 [    55   2794]]
